In [1]:
import logging
from gensim import utils
from gensim import models
from gensim.corpora.textcorpus import TextCorpus, strip_multiple_whitespaces, remove_stopwords
from gensim.models.phrases import Phrases, Phraser
from pprint import pprint

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

import pandas as pd
import psutil
import sys
import os

###
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
###



logger = logging.getLogger(__name__)

#json_file = './head.json'
json_file = './2000.json'
#json_file = './reviews_Books_5.json'

bigram_model = None

def memory_usage_psutil():
    # return the memory usage in MB
    import psutil
    process = psutil.Process(os.getpid())
    mem = process.memory_info()[0] / float(2 ** 20)
    return mem

def to_unicode(text, encoding='utf8', errors='strict'):
    """ adapted from TextCorpus.lower_to_unicode (just unicode)

    """
    return utils.to_unicode(text, encoding, errors)

wn_lemm = WordNetLemmatizer()
def lemmatize(tokens):
    return [wn_lemm.lemmatize(token) for token in tokens]

def lemmatize2(tokens):
    r = []

    for token in tokens:
        lem = wn.morphy(token)
        if lem:
            r.append(lem)
        else:
            r.append(token)

    return r

def apply_bigrams(tokens):
    if bigram_model is None:
        logger.error("no bigram model")

    return bigram_model[tokens]

def lowercase(tokens):
    """Remove stopwords using list from `gensim.parsing.preprocessing.STOPWORDS`.
    """
    return [token.lower() for token in tokens]

def split_tokenizer(text):
    for token in text.split():
        yield token

class CorpusTest(TextCorpus):
    #stopwords = set('for a of the and to in on'.split())
    filename = json_file

    def get_texts(self):
        reader = pd.read_json(self.filename, lines=True, chunksize=1)
        for chunk in reader:
            # more input modifications?
            yield self.preprocess_text(chunk.reviewText.values[0])
            #yield utils.to_unicode(chunk.reviewText.values[0]).split()

        #for doc in self.getstream():
        #    yield [word for word in utils.to_unicode(doc).lower().split() if word not in self.stopwords]

    def __len__(self):
        self.length = sum(1 for _ in self.get_texts())
        return self.length

    def gen_bigram_model(self):
        bigrams = Phrases(iter(self.get_texts()))
        print(memory_usage_psutil())
        return Phraser(bigrams)


corpus = CorpusTest(input="not_used",
                    character_filters=[to_unicode, strip_multiple_whitespaces],
                    token_filters=[lemmatize2, lowercase, remove_stopwords],
                    tokenizer=split_tokenizer)

# create bigram model
bigram_model = corpus.gen_bigram_model()

bicorpus = CorpusTest(input="not_used",
                      character_filters=[to_unicode, strip_multiple_whitespaces],
                      token_filters=[lemmatize2, lowercase, remove_stopwords, apply_bigrams],
                      tokenizer=split_tokenizer)


lda_model = models.LdaModel(bicorpus, id2word=bicorpus.dictionary, num_topics=15)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

print("done")




C:\Users\elli\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


731.13671875
[(0,
  '0.015*"life" + 0.011*"ha" + 0.010*"woman" + 0.010*"love" + 0.006*"child" + '
  '0.006*"family" + 0.004*"help" + 0.004*"mother" + 0.004*"father" + '
  '0.004*"man"'),
 (1,
  '0.006*"la" + 0.003*"que" + 0.003*"y" + 0.002*"el" + 0.002*"story" + '
  '0.002*"al" + 0.002*"wa" + 0.002*"ha" + 0.002*"love" + 0.002*"miller"'),
 (2,
  '0.008*"ha" + 0.007*"world" + 0.006*"-" + 0.004*"work" + 0.003*"--" + '
  '0.003*"art" + 0.003*"book" + 0.003*"create" + 0.003*"human" + 0.003*"like"'),
 (3,
  '0.003*"love" + 0.003*"like" + 0.003*"book" + 0.003*"ha" + 0.003*"card" + '
  '0.003*"chess" + 0.002*"--" + 0.002*"manga" + 0.002*"adventure" + '
  '0.002*"read"'),
 (4,
  '0.048*"book" + 0.009*"it\'s" + 0.008*"read" + 0.008*"like" + 0.007*"buy" + '
  '0.006*"book." + 0.006*"want" + 0.006*"great" + 0.006*"good" + 0.005*"ha"'),
 (5,
  '0.034*"book" + 0.008*"use" + 0.006*"ha" + 0.006*"author" + 0.005*"need" + '
  '0.004*"information" + 0.004*"work" + 0.004*"chapter" + 0.004*"good" + '
  '0.

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Compute Perplexity
#print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Computation of Coherence Score
#coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=bicorpus.dictionary, coherence='c_v')
#coherence_lda = coherence_model_lda.get_coherence()
#print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, bicorpus, bicorpus.dictionary)
#vis


#saving the vis.:

vis = pyLDAvis.gensim.prepare(lda_model, bicorpus, bicorpus.dictionary)
pyLDAvis.save_html(vis, "test.html")


print("done too")


done too
